<a href="https://colab.research.google.com/github/AfifaMasood/AfifaMasood/blob/main/Copy_of_Resnet%2Bcropped_images%2Byolo%2Baug_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import time
import shutil
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Custom Dataset class to extract labels from filenames
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_shoplifting=None, max_normal=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.corrupt_images = []
        shoplifting_count, normal_count = 0, 0

        print(f"🔄 Loading dataset from: {root_dir}")

        for subdir, _, files in os.walk(root_dir):
            for file in sorted(files):
                if file.endswith(".jpg"):
                    file_path = os.path.join(subdir, file)
                    if os.path.getsize(file_path) == 0:
                        self.corrupt_images.append(file)
                        continue
                    try:
                        with Image.open(file_path) as img:
                            img.verify()
                    except (UnidentifiedImageError, IOError):
                        self.corrupt_images.append(file)
                        continue

                    start_time = time.time()
                    label = None

                    if file.startswith("Shoplifting_aug_") or file.startswith("Shoplifting-"):
                        label = 'Shoplifting'
                        shoplifting_count += 1
                    elif file.startswith("Normal_aug_") or file.startswith("Normal-"):
                        label = 'Normal'
                        normal_count += 1

                    if time.time() - start_time > 10:
                        print(f"Skipping {file} due to timeout.")
                        continue

                    if label:
                        self.image_paths.append(file_path)
                        self.labels.append(label)

        self.label_map = {'Shoplifting': 1, 'Normal': 0}
        self.labels = [self.label_map.get(label, 0) for label in self.labels]

        print(f"✅ Dataset loaded: {len(self.image_paths)} images found (Skipped {len(self.corrupt_images)} corrupt images)")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        try:
            image = Image.open(img_name).convert("RGB")
        except (UnidentifiedImageError, IOError):
            return None, None
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define dataset paths separately
train_dataset_path = "/content/drive/MyDrive/augmented_dataset_17000/train"
test_dataset_path = "/content/drive/MyDrive/augmented_dataset/test"

# Load training dataset with limits
train_dataset = CustomDataset(root_dir=train_dataset_path, transform=transform)

# Load test dataset with all images (limit set to 5000)
test_dataset = CustomDataset(root_dir=test_dataset_path, transform=transform)

# Filter None values
train_dataset.image_paths, train_dataset.labels = zip(*[(img, lbl) for img, lbl in zip(train_dataset.image_paths, train_dataset.labels) if lbl is not None])
test_dataset.image_paths, test_dataset.labels = zip(*[(img, lbl) for img, lbl in zip(test_dataset.image_paths, test_dataset.labels) if lbl is not None])

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load and modify ResNet34
print("⏳ Loading ResNet34 model...")
model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
print("✅ ResNet34 model loaded and modified.")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
print("\n🚀 Starting Training...\n")
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)
        outputs = model(data)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Save trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/resnet34_shoplifting_aug_image1.pth")
print("\n🎉 Training complete. Model saved. 🎉")


🔄 Loading dataset from: /content/drive/MyDrive/augmented_dataset_17000/train
✅ Dataset loaded: 34000 images found (Skipped 0 corrupt images)
🔄 Loading dataset from: /content/drive/MyDrive/augmented_dataset/test
✅ Dataset loaded: 5000 images found (Skipped 0 corrupt images)
⏳ Loading ResNet34 model...


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 89.7MB/s]


✅ ResNet34 model loaded and modified.

🚀 Starting Training...

Epoch [1/10], Loss: 0.3483
Epoch [2/10], Loss: 0.1464
Epoch [3/10], Loss: 0.1093
Epoch [4/10], Loss: 0.0765
Epoch [5/10], Loss: 0.0611
Epoch [6/10], Loss: 0.0569
Epoch [7/10], Loss: 0.0549
Epoch [8/10], Loss: 0.0482
Epoch [9/10], Loss: 0.0371
Epoch [10/10], Loss: 0.0343

🎉 Training complete. Model saved. 🎉


In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation during inference
        for x, y in loader:
            if x is None or y is None:  # Skip None batches
                continue

            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += y.size(0)

    accuracy = float(num_correct) / num_samples * 100
    return accuracy

# Load and check train accuracy
model.load_state_dict(torch.load('/content/drive/MyDrive/resnet34_shoplifting_aug_image1.pth', map_location=device))
model.to(device)

# Train Accuracy
print('Train Accuracy:')
train_accuracy = check_accuracy(train_loader, model)
print(f'Train Accuracy: {train_accuracy:.2f}%')

# Test Accuracy
print('Test Accuracy:')
test_accuracy = check_accuracy(test_loader, model)
print(f'Test Accuracy: {test_accuracy:.2f}%')


<ipython-input-3-ca0c8090ef7c>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/resnet34_shoplifting_aug_image1.pth

Train Accuracy:
Train Accuracy: 99.24%
Test Accuracy:
Test Accuracy: 97.20%
